In [1]:
# Google Sheets Credentials
# import required libraries
import pandas as pd
import os.path, urllib.request, json, requests

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

## Google Sheets API - Inputs
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']

MH_SPREADSHEET_ID = '1xWFQgV3T2NOvfN_I8kvFxWHbH9i240rp2UFi2SEi6Zc'
MH_RANGE_NAME = 'Data!A1:M'

MHDATA_ID = '1KeW44qsH7WCsuXuQHrZs7lG6yiMirT0PAKW_axtpv2c'
MHDATA_RANGE_NAME1 = 'MP!A2:B'
MHDATA_RANGE_NAME2 = 'MP!D2:E'
MHDATA_RANGE_NAME3 = 'Snipe!A2:E'

creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = build('sheets', 'v4', credentials=creds)

In [2]:
# Load MH DB spreadsheet into py using .get
# Call the Sheets API to .get value
sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=MH_SPREADSHEET_ID,range=MH_RANGE_NAME).execute()
values = result.get('values', [])
value_input_option = 'USER_ENTERED'

# Do you want to force an update?
force_update = False

# Get values from MP sheet
gsheet_id = sheet.values().get(spreadsheetId=MHDATA_ID,range='MP!A:A').execute().get('values', [])

#### ITEMS ####
# Create pd dataframe, only select tradable items
df = pd.DataFrame(values[1:], columns=values[0])
df.set_index(['ID'], drop=False, inplace=True)
df = df[df['Tradable'] == 'Yes'][['ID','Name']]

# Check if highest ID is latest. If False, add ID & Name of all items.
if df.index[-1] > gsheet_id[-1][0] or df.shape[0] < len(gsheet_id) - 1:
    result = sheet.values().update(spreadsheetId=MHDATA_ID,range=MHDATA_RANGE_NAME1, valueInputOption=value_input_option, body={'values': df.values.tolist()}).execute()
    print('New items detected, updating Gsheet')
elif force_update == True:
    result = sheet.values().update(spreadsheetId=MHDATA_ID,range=MHDATA_RANGE_NAME1, valueInputOption=value_input_option, body={'values': df.values.tolist()}).execute()
    print('Forced update, updating items in Gsheet')
else:
    print('No new items')

#### MICE ####
# Import mice data from API
req = urllib.request.Request(
    url='https://api.mouse.rip/mice', 
    headers={'User-Agent': 'Mozilla/5.0'}
)
with urllib.request.urlopen(req) as url: data = json.load(url)

# Convert data to pd
mdf = pd.json_normalize(data)[['id', 'name', 'group', 'subgroup']].values.tolist()
mvalues = sheet.values().get(spreadsheetId=MHDATA_ID,range='Snipe!A:E').execute().get('values', [])

# Compare Sheets latest mice ID with API, if not the same, update the spreadsheet
if mdf[-1][0] > int(mvalues[-1][0]):
    result = sheet.values().update(spreadsheetId=MHDATA_ID,range=MHDATA_RANGE_NAME3, valueInputOption=value_input_option, body={'values': mdf}).execute()
    print('New mice detected, updating Gsheet')
elif force_update == True:
    result = sheet.values().update(spreadsheetId=MHDATA_ID,range=MHDATA_RANGE_NAME3, valueInputOption=value_input_option, body={'values': mdf}).execute()
    print('Forced update, updating mice in Gsheet')
else:
    print('No new mice')

df['Date'], df['Price'], df['GoldChange'] = None, 0, 0

In [4]:
# Import MP price from Markethunt API
id = df['ID'].values.tolist()
f_url = 'https://api.markethunt.win/items/'
url = [f_url + s for s in id]
latest_date = json.loads(requests.get('https://api.markethunt.win/items/114').text).get('market_data')[-1]['date'] # Gets Latest Date from item 114 (SB)

for i in range(len(id)):
    market_data = json.loads(requests.get(url[i]).text).get('market_data')
    if market_data != []: # if data is not null
        if market_data[-1]['date'] < latest_date: # if last data date is not today, use it anyways
            df.iat[i,2], df.iat[i,3], df.iat[i,4] = market_data[-1]['date'], market_data[-1]['price'], 0 # Date, Price, GoldChange
        else:
            try:
                df.iat[i,2], df.iat[i,3], df.iat[i,4] = market_data[-1]['date'], market_data[-1]['price'], market_data[-1]['price'] - market_data[-2]['price']
            except IndexError: #if list out of range (only 1 date data), will return vGC as 0
                df.iat[i,2], df.iat[i,3], df.iat[i,4] = market_data[-1]['date'], market_data[-1]['price'], 0
    else: # if data is null
        df.iat[i,2], df.iat[i,3], df.iat[i,4] = None, 0, 0

In [5]:
# Add MP Price & Date Updated into Spreadsheet by using .update
data1 = df.loc[:,['Date']].to_numpy().tolist()
data2 = df.loc[:,['Price','GoldChange']].to_numpy().tolist()
batch_update_values_request_body = {
    'value_input_option': 'USER_ENTERED',
    'data': [{
        'range': "MP!D2:D",
        'values': data1
    },
    {
        'range': "MP!F2:G",
        'values': data2
    }]
}
result = sheet.values().batchUpdate(spreadsheetId=MHDATA_ID, body=batch_update_values_request_body).execute()